# Rozszerzenie głównego zbioru danych o informacje o pobliskich obiektach medycznych
Dane są niskiej jakości i duża część tras nie wygląda, jakby była rzeczywistą trasą karetki na służbie.
Aby odfiltrować niewłaściwe rekordy dodajemy w tym notatniku dane z OpenStreetMap o małopolskich obiektach medycznych, a później wykorzystujemy je do zweryfikowania, czy punkt początkowy i końcowy trasy znajduje się przy obiekcie medycznym.

Poza tym dodajemy również informacje o tym, czy postoje karetki są przy obiektach medycznych, a także poprawiamy ich jakość. W poprzednim notatniku dodano punkty reprezentatywne klastrów, teraz sprawdzamy, czy nie są one blisko punktów początkowego i końcowego trasy. Jeśli tak, to zostają usunięte

In [1]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
from shapely.geometry import Point
import branca.colormap as cm
import folium
import numpy as np
from sklearn.neighbors import BallTree
import swifter

data_path = r"X:\dane_karetki\karetki_klastry.parquet"
medical_facilities_path = r"X:\dane_karetki\hospitals.csv"

In [2]:
if 'gdf' not in globals():
    # automatycznie odczyta geometrię
    gdf = gpd.read_parquet(data_path)

    gdf = gdf.sort_values(by='Czas wezwania', ascending=True)

    # Próbkowanie do 10% danych
    # gdf = gdf.sample(frac=0.1, random_state=42) # Dodaj random_state dla powtarzalności
    gdf = gdf.reset_index(drop=True)

if gdf.crs is None:
     gdf = gdf.set_crs(3857, allow_override=True)
elif gdf.crs.to_epsg() != 3857:
     gdf = gdf.to_crs(3857)

gdf['dense_cluster'] = gdf['dense_cluster_xy'].apply(
    lambda lst: [Point(xy) for xy in lst] if lst is not None else None
)

In [3]:
gdf = gdf.dropna(subset=['Line']).copy()

In [4]:
medical_facilities = pd.read_csv(medical_facilities_path)
medical_facilities['geometry'] = medical_facilities['geometry'].apply(wkt.loads)
medical_facilities = gpd.GeoDataFrame(medical_facilities, geometry='geometry').set_crs(3857, allow_override=True)
medical_facilities.head()

,name,geometry,amenity
0,Małopolski Ośrodek Medycyny Pracy,POINT (2220884.677 6457589.184),clinic
1,NaN,POINT (2220133.616 6462247.385),doctors
2,Szpital Zakonu Bonifratrów Świętego Jana Grandego,POINT (2220287.994 6454510.007),hospital
3,Klinika Janeczko - medycyna estetyczna i derma...,POINT (2219663.592 6455679.499),doctors
4,"Laboratorium Protetyki Stomatologicznej ""Top D...",POINT (2219116.211 6457295.488),doctors


## Znalezienie najbliższych obiektów medycznych dla punktu początkowego i końcowego trasy

In [5]:
first_points = gdf['Line'].apply(lambda line: Point(line.coords[0]))
last_points = gdf['Line'].apply(lambda line: Point(line.coords[-1]))

first_points_gs = gpd.GeoSeries(first_points, crs=3857)
last_points_gs = gpd.GeoSeries(last_points, crs=3857)

first_points_gdf = gpd.GeoDataFrame(geometry=first_points_gs)
last_points_gdf = gpd.GeoDataFrame(geometry=last_points_gs)

first_nearest = gpd.sjoin_nearest(
    first_points_gdf, medical_facilities, how='left', max_distance=300, distance_col='dist'
)

last_nearest = gpd.sjoin_nearest(
    last_points_gdf, medical_facilities, how='left', max_distance=300, distance_col='dist'
)

In [6]:
first_nearest['has_nearby_medical'] = first_nearest['index_right'].notna()
first_nearest = (
    first_nearest
    .rename(columns={
        'name': 'nearest_medical_name',
        'amenity': 'nearest_medical_type'
    })
    .drop(columns=['dist', 'index_right']) 
)

last_nearest['has_nearby_medical'] = last_nearest['index_right'].notna()
last_nearest = (
    last_nearest
    .rename(columns={
        'name': 'nearest_medical_name',
        'amenity': 'nearest_medical_type'
    })
    .drop(columns=['dist', 'index_right']) 
)

In [7]:
last_nearest

,geometry,nearest_medical_name,nearest_medical_type,has_nearby_medical
0,POINT (2219039.546 6459501.092),SOR,clinic,True
1,POINT (2218562.664 6460757.073),NaN,NaN,False
2,POINT (2178465.71 6492874.148),Nowy Szpital Olkusz,hospital,True
3,POINT (2303392.478 6355622.912),NaN,NaN,False
4,POINT (2223962.303 6461912.637),NaN,NaN,False
...,...,...,...,...
152883,POINT (2336760.214 6449093.28),Zespół Przychodni Specjalistycznych,doctors,True
152884,POINT (2273239.981 6440446.427),Oddział Położniczo-Ginekologiczny,clinic,True
152885,POINT (2261673.573 6411402.455),NaN,NaN,False
152886,POINT (2232003.479 6448125.173),NaN,NaN,False


In [8]:
gdf.shape

(152888, 17)

Tyle początkowych punktów tras karetek z naszego zbioru jest w obrębie 300m od obiektu medycznego

In [9]:
# policz % true w zmiennej first_within_300m
first_within_300m_percentage = (first_nearest['has_nearby_medical'].sum() / len(first_nearest)) * 100
first_within_300m_percentage

np.float64(53.146747946209)

Tyle punktów końcowych tras karetek z naszego zbioru jest w obrębie 300m od obiektu medycznego

In [10]:
last_within_300m_percentage = (last_nearest['has_nearby_medical'].sum() / len(last_nearest)) * 100
last_within_300m_percentage

np.float64(52.310187849929356)

In [11]:
gdf_filtered = gdf.loc[first_nearest['has_nearby_medical']].copy()
gdf_filtered2 = gdf.loc[last_nearest['has_nearby_medical']].copy()

In [12]:
def plot_route(row, cluster_info):
    # Ensure geometry is in EPSG:4326
    line = row['Line']
    if hasattr(line, 'crs'):
        # If line is a GeoSeries, ensure correct CRS
        if getattr(line, 'crs', None) != 4326:
            line = gpd.GeoSeries([line], crs=line.crs).to_crs(4326).iloc[0]
    else:
        # If not, assume gdf.crs and convert
        line = gpd.GeoSeries([line], crs=gdf.crs).to_crs(4326).iloc[0]
    coords = [(y, x) for x, y in line.coords]
    colormap = cm.linear.viridis.scale(0, len(coords) - 2)
    m = folium.Map(location=coords[0], zoom_start=13)

    for i in range(len(coords) - 1):
        segment = [coords[i], coords[i + 1]]
        color = colormap(i)
        folium.PolyLine(segment, color=color, weight=5).add_to(m)

    folium.Marker(location=coords[0], popup='Start', icon=folium.Icon(color='green', icon='play')).add_to(m)
    folium.Marker(location=coords[-1], popup='End', icon=folium.Icon(color='red', icon='stop')).add_to(m)

    # Convert reference point to EPSG:4326 if needed
    lat = row['Szerokość geograficzna']
    lon = row['Dlugość geograficzna']
    folium.Marker(location=[lat, lon], popup='Extra point', icon=folium.Icon(color='blue', icon='info-sign')).add_to(m)

    has_cluster, dense_points = cluster_info
    if has_cluster and len(dense_points) > 0:
        dense_points = [Point(pt) for pt in dense_points]
        dense_points_gs = gpd.GeoSeries(dense_points, crs=gdf.crs).to_crs(4326)
        for pt in dense_points_gs:
            folium.CircleMarker(
                location=[pt.y, pt.x],
                radius=6,
                color='orange',
                fill=True,
                fill_color='orange',
                fill_opacity=0.7,
                popup='Dense cluster'
            ).add_to(m)

    colormap.caption = 'Order of points'
    colormap.add_to(m)
    return m

In [13]:
sample = gdf_filtered2.sample(1).iloc[0]
display(plot_route(sample, (False, [])))

In [14]:
# gdf_filtered = gdf.to_crs(3857)

def all_points_within_m_proj(line, threshold=200):
    coords = np.array(line.coords)
    start = coords[0]
    dists = np.linalg.norm(coords - start, axis=1)
    return np.all(dists <= threshold)

In [15]:
result = gdf_filtered['Line'].apply(all_points_within_m_proj)
round(float(result.sum() / len(gdf) * 100), 2)

0.0

In [16]:
all_points_within_m_proj(sample['Line'], threshold=600)

np.False_

In [17]:
sample

Czas wezwania                                                                      2022-01-21 14:24:50
Czas wyjazdu ZRM                                                                   2022-01-21 14:25:46
Czas powrotu ZRM                                                                   2022-01-21 15:24:12
Powód wezwania                                                                             Ból brzucha
Kod pilności                                                                                         2
Dlugość geograficzna                                                                         20.973255
Szerokość geograficzna                                                                       50.015141
Identyfikator pojazdu                                                                         KT 6699E
Rodzaj wyjazdu 0- na sygnale, 1 -zwykly                                                              1
Typ zespolu                                                              

In [18]:
gdf['start_has_nearby_medical'] = first_nearest['has_nearby_medical']
gdf['start_nearest_medical_name'] = first_nearest['nearest_medical_name']
gdf['start_nearest_medical_type'] = first_nearest['nearest_medical_type']

gdf['end_has_nearby_medical'] = last_nearest['has_nearby_medical']
gdf['end_nearest_medical_name'] = last_nearest['nearest_medical_name']
gdf['end_nearest_medical_type'] = last_nearest['nearest_medical_type']

## Odfiltrowanie klastrów, które znajdują się w pobliżu początku i końca linestringu

In [19]:
def filter_clusters(row):
    clusters = row['dense_cluster']
    if clusters is None:
        return None
    start = Point(row['Line'].coords[0])
    end = Point(row['Line'].coords[-1])
    filtered = [
        pt for pt in clusters
        if (pt.distance(start) > 300) and (pt.distance(end) > 300)
    ]
    return filtered if filtered else None

In [20]:
gdf['dense_cluster'] = gdf.apply(filter_clusters, axis=1)

In [35]:
gdf['has_cluster'] = gdf['dense_cluster'].apply(lambda x: x is not None and len(x) > 0)

## Znalezienie postojów przy obiektach medycznych

In [26]:
medical_points = medical_facilities['geometry'].values
med_coords = np.array([[pt.x, pt.y] for pt in medical_points])
tree = BallTree(med_coords, metric='euclidean')

In [27]:
def clusters_near_medical(row):
    clusters = row['dense_cluster']
    if clusters is None:
        return None
    cluster_coords = np.array([[pt.x, pt.y] for pt in clusters])
    dists, _ = tree.query_radius(cluster_coords, r=300/111320, return_distance=True)  # 111320m na stopień
    return [len(d) > 0 for d in dists]

In [29]:
gdf['breaks_nearby_medical'] = gdf.swifter.apply(clusters_near_medical, axis=1)

Pandas Apply:   0%|          | 0/152888 [00:00<?, ?it/s]

In [33]:
gdf.columns

Index(['Czas wezwania', 'Czas wyjazdu ZRM', 'Czas powrotu ZRM',
       'Powód wezwania', 'Kod pilności', 'Dlugość geograficzna',
       'Szerokość geograficzna', 'Identyfikator pojazdu',
       'Rodzaj wyjazdu 0- na sygnale, 1 -zwykly', 'Typ zespolu',
       'Określenie wieku pacjenta 0- dziecko, 1 - dorosly', 'ID_GPS', 'Line',
       'Line_time', 'has_cluster', 'dense_cluster_xy', 'dense_cluster',
       'start_has_nearby_medical', 'start_nearest_medical_name',
       'start_nearest_medical_type', 'end_has_nearby_medical',
       'end_nearest_medical_name', 'end_nearest_medical_type',
       'breaks_nearby_medical'],
      dtype='object')

In [36]:
gdf[['has_cluster', 'dense_cluster', 'breaks_nearby_medical']]

,has_cluster,dense_cluster,breaks_nearby_medical
0,False,None,None
1,False,None,None
2,False,None,None
3,False,None,None
4,False,None,None
...,...,...,...
152883,True,"[POINT (2341406.310926805 6448402.046758858), ...","[False, False]"
152884,False,None,None
152885,False,None,None
152886,False,None,None


## Zapisanie przefiltrowanych danych do pliku

In [37]:
def to_tuples(points):
    if points is None:
        return None
    return [(p.x, p.y) for p in points]

gdf['dense_cluster_xy'] = gdf['dense_cluster'].apply(to_tuples)
gdf.drop(columns=['dense_cluster']) \
    .to_parquet(r"X:\dane_karetki\trasy_karetek_obiekty_klastry.parquet", engine='pyarrow', index=False)